In [1]:
#Version 1.0

from bokeh.io import push_notebook, show, output_notebook, save
from bokeh.plotting import figure, output_file, show
from bokeh.models import LinearAxis, Range1d, HoverTool
from bokeh.layouts import column, row, gridplot, layout
from bokeh.models import ColumnDataSource, Div
from bokeh.models.widgets import Panel, Tabs
import bokeh

from itertools import cycle
import dmyplant2
from dmyplant2.dPlot import bokeh_chart, datastr_to_dict, expand_cylinder, shrink_cylinder, load_pltcfg_from_excel,show_val_stats
import arrow


import pandas as pd
import numpy as np
import math
import traceback
import matplotlib
import sys
import warnings
import logging
import datetime
import pytz
import os 

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

logging.basicConfig(
    filename='dmyplant.log',
    filemode='w',
    format='%(asctime)s %(levelname)s, %(message)s',
    level=logging.INFO
)
logging.captureWarnings(True)
hdlr = logging.StreamHandler(sys.stdout)
logging.getLogger().addHandler(hdlr)

In [4]:
dmyplant2.cred()
mp = dmyplant2.MyPlant(0)
from urllib3.exceptions import NewConnectionError
import urllib
import socket

class myEngine(dmyplant2.Engine):
    @ property
    def dash(self):
        _dash = dict()
        _dash['Name'] = self.Name
        _dash['serialNumber'] = self.serialNumber
        _dash['Site'] = self.get_property('IB Site Name') 
        _dash['Engine ID'] = self.get_property('Engine ID')
        _dash['Design Number'] = self.get_property('Design Number')
        _dash['Engine Type'] = self.get_property('Engine Type')
        _dash['Engine Version'] = self.get_property('Engine Version')
        _dash['Gas type'] = self.get_property('Gas Type')
        _dash['Country'] = self.get_property('Country')
        _dash['Engine OPH'] = self.Count_OpHour
        _dash['Engine Starts'] = self.Count_Starts
        _dash['Part OPH'] = self.oph_parts
        _dash['Part Starts'] = self.starts_parts
        _dash['P_nom'] = self.Pmech_nominal
        _dash['BMEP'] = self.BMEP
        _dash['LOC'] = self.get_dataItem(
            'RMD_ListBuffMAvgOilConsume_OilConsumption')
        return _dash


try:
    #Load first sheet validation
    #Iterate over validation numbers taken from first sheet
        #Call dashboard


        #Eventually sort engines by operating hours, but only if markers work: https://stackoverflow.com/questions/59216329/how-to-export-the-pandas-conditional-formatting-to-an-excel-file
        #Save dashboard in output file, sheetname=Validation number

    df_val_info=pd.read_excel('Input_overview_progress_validations.xlsx', sheet_name='Validations')#, usecols=['Variable', 'Value']) #loading of relevant excel sheet in DataFrame
    df_val_info.dropna(subset=['Field validation number'], inplace=True)
    cols = [c for c in df_val_info.columns if (c.lower()[:10] == 'inspection' or c=='Field validation number')]
    df_val_info=df_val_info[cols]
   
    df_val_info = df_val_info.astype({'Field validation number':'int'}) #little workaround to avoid .0 at the end
    df_val_info = df_val_info.astype({'Field validation number':'str'}) 

    df_val_info.set_index('Field validation number', inplace=True)

    #df_val_info=df_val_info.apply(pd.to_numeric, errors='coerce')
    with pd.ExcelWriter(f'Output_overview_progress_validations.xlsx.xlsx') as writer:
        for val_nr in df_val_info.index:
            print('Loading information for validation '+val_nr)
            dval=dmyplant2.Validation.load_def_excel('Input_overview_progress_validations.xlsx', val_nr, mp) #Loading of validation engine data from excel
            vl = dmyplant2.Validation(mp, dval, lengine=myEngine, cui_log=False)

            logging.info('Engine properties for validation '+val_nr+' loaded')

            dash=vl.dashboard
            #dash.loc[:, ['next insp', 'h next insp', '%next insp', '%overall']] = np.nan #replaced by lines below to be compatible with older versions of Python
            dash['next insp'] = np.nan
            dash['h next insp'] = np.nan
            dash['%next insp'] = np.nan
            dash['%overall'] = np.nan

            inspections=df_val_info.loc[[val_nr]]
            inspections=inspections[inspections.columns[~inspections.isnull().any()]].values.tolist()[0] #filter out nan columns
            if len(inspections)==0: #check if inspection steps defined
                raise ValueError('Error! Please specify inspection steps for validation '+val_nr+'!')
            for idx in dash.index:
                oph_parts=dash.loc[idx, 'Part OPH']
                next_inspection = min([i for i in inspections if i > oph_parts], default='finished')

                if next_inspection!='finished':
                    last_inspection = max([i for i in inspections if i < oph_parts], default=0)
                    proz_next_inspection=((oph_parts-last_inspection)/(next_inspection-last_inspection)*100).astype(int)
                    try:
                        proz_next_inspection=proz_next_inspection.astype(int)
                    except:
                        pass
                    h_next_inspection=next_inspection-oph_parts
                else:
                    proz_next_inspection=100
                    h_next_inspection=-1
                
                proz_overall=min(100, oph_parts/max(inspections)*100)
                try:
                    proz_overall=proz_overall.astype(int)
                except:
                    pass

                dash.loc[idx, ['next insp', 'h next insp', '%next insp', '%overall']] = [next_inspection, h_next_inspection, proz_next_inspection, proz_overall]
 
            dash.to_excel(writer, val_nr, float_format="%.3f")

except Exception as e:
    print(e)
    if str(e)=="'Engine' object has no attribute 'asset'":
        print ('Possible cause: No internet connection')
    traceback.print_tb(e.__traceback__)
  
finally:
    print('\nFinished! Find output in: Output_overview_progress_validations.xlsx')

    hdlr.close()
    logging.getLogger().removeHandler(hdlr)



Loading information for validation 204
50000it [00:01, 29620.50it/s]
50000it [00:00, 304393.87it/s]
Loading information for validation 265
50000it [00:00, 314600.48it/s]
50000it [00:00, 310482.66it/s]
50000it [00:00, 291794.31it/s]
Loading information for validation 301
50000it [00:00, 295640.43it/s]

Finished! Find output in: Output_overview_progress_validations.xlsx
